In [115]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.support.ui import Select
import pandas as pd
import re
from time import sleep
from datetime import datetime,timedelta
from pytz import timezone

#DB 접속 정보 -> 붙여넣으시면 됩니다.
#########################################
import pyodbc
server = 'sap-sql.database.windows.net' 
database = 'sap-master' 
username = 'smartmind' 
password = 'qazWSXedc!@#' 
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn.autocommit = True
cursor = cnxn.cursor()
cursor.execute("set language Korean")
#########################################

In [116]:
path_driver = '../chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('--disable-extensions')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument("--disable-dev-shm-usage")
options.add_argument("lang=en_US")
driver = webdriver.Chrome(path_driver, options=options)

now = datetime.now(timezone('Asia/Seoul'))
print(now)
driver.get("https://www.opinet.co.kr/glopcoilSelect.do")
sleep(15)
today = datetime.now() - timedelta(days=1)
today_year = today.strftime("%Y")
today_month = int(today.strftime("%m"))
today_day = int(today.strftime("%d")) - 1

Select(driver.find_element_by_xpath('//*[@id="STA_Y"]')).select_by_visible_text("2008")
Select(driver.find_element_by_xpath('//*[@id="STA_M"]')).select_by_visible_text("1")
Select(driver.find_element_by_xpath('//*[@id="STA_D"]')).select_by_visible_text("1")
Select(driver.find_element_by_xpath('//*[@id="END_Y"]')).select_by_visible_text(today_year)
Select(driver.find_element_by_xpath('//*[@id="END_M"]')).select_by_visible_text(str(today_month))
Select(driver.find_element_by_xpath('//*[@id="END_D"]')).select_by_visible_text(str(today_day))
sleep(1)
Select = driver.find_element_by_xpath('//*[@id="glopcoilSelect"]/span')
driver.execute_script("arguments[0].click();",Select)
sleep(15)
soup = BeautifulSoup(driver.page_source, 'html.parser')
tableList = soup.select('#tbody2 > tr')
time = []
dubai = []
brent = []
wit = []
oil_data = pd.DataFrame() #columns=["time", "dubai", "brent", "WTI"])


for tablerow in tableList :
    dataList = tablerow.select('td')
    if str(dataList[0].text) == "조회결과가 없습니다." :
        break;

    timestep = datetime.strptime(str(dataList[0].text),"%y년%m월%d일")
    time.append(timestep.strftime("%Y-%m-%d"))
    dubai.append(re.sub("\n|\t","",str(dataList[1].text),0))
    brent.append(re.sub("\n|\t","",str(dataList[2].text),0))
    wit.append(re.sub("\n|\t","",str(dataList[3].text),0))

oil_data['날짜'] = time
oil_data['dubai'] = dubai
oil_data['brent'] = brent
oil_data['WIT'] = wit

driver.quit()

2021-07-13 10:19:41.552265+09:00


In [112]:
oil_data

,날짜,dubai,brent,WIT
0,2008-01-02,89.29,97.84,99.62
1,2008-01-03,92.03,97.60,99.18
2,2008-01-04,92.29,96.79,97.91
3,2008-01-07,91.54,94.39,95.09
4,2008-01-08,90.12,95.54,96.33
...,...,...,...,...
3483,2021-06-30,72.87,75.13,73.47
3484,2021-07-01,73.85,75.84,75.23
3485,2021-07-02,74.09,76.17,75.16
3486,2021-07-05,74.68,77.16,-


In [113]:
def insertData(df,cursor,cnxn) :
   # Insert Dataframe into SQL Server:
   for index, row in df.iterrows():
        cursor.execute("INSERT INTO OILPRICE (datetime,dubai,brent,WIT) values(?,?,?,?)", row.날짜, row.dubai,row.brent,row.WIT)

In [114]:
cursor.execute("truncate table OILPRICE")
if len(oil_data) > 0 :
    insertData(oil_data,cursor,cnxn)

